# 앙상블 학습 방법
## 1. 용어
### 기본 학습기 Base Learner
- 개별적으로 학습된 모델
- 약한 학습기라고도 함
### 강한 학습기 Strong Learner
-  여러 개의 약한 학습기를 결합하여 만든 모델
### 배깅 Bagging
- 여러 개의 모델을 병렬로 학습시키고, 그 결과를 결합하는 방법
- 각 모델은 데이터의 부트스트랩 샘플을 사용하여 학습
- 대표적으로 랜덤 포레스트Random Forest
### 부스팅 Boosting
- 여러 개의 모델을 순차적으로 학습시킴
- 각 모델은 이전 모델의 오차를 줄이는 데 집중
- 대표적으로 에이다부스트 Adaboost, 그레이디언트 부스팅 머신GBM
### 스태킹 Stacking
- 여러 개의 다른 모델의 예측 결과를 모아서 새로운 모델(메타 모델)을 학습시킴
메타 모델은 개별 모델의 예측 결과를 입력으로 받아 최종 예측으로 만듬
- 다양한 모델 결합 :  Logistic Regression, SVM, Random Forest 등 다양한 모델을 결합하여 더 강력한 예측 가능
## 2. 개념
- 여러 개의 기본 모델(개별 예측기 또는 기본 학습기)을 결합하여 더 강력한 예측 모델을 만드는 방법
- 개별 모델이 가진 약점을 보완하고, 전체적인 성능을 향상시키는 것이 목적
## 3. 배깅 : 부트스트랩 샘플링을 통한 분류 앙상블
- 배깅은 부트스트랩 샘플링을 사용하여 여러 개의 모델을 학습시키고, 이 모델들의 예측을 평균 내거나 다수결 투포를 통해 최종 예측을 생성
### 4.1 배깅 알고리즘의 작동 방식
- 부트스트랩 샘플링 : 원본 데이터셋에서 중복을 허용하여 여러 개의 샘플 뽑음
- 각 샘프을 이용하여 모델 학습
- 모든 모델의 예측을 결합하여 최종 예측 생성
## 5. 약한 학습기를 이용한 에이다부스트
- 에이다부스트Adaboost는 약한 학습기를 연속적으로 학습시켜 예측 성능을 향상시키는 부스팅 기법
### 5.1 부스팅 작동 원리
- 첫번째 모델을 학습시킨다
- 모델의 오차를 분석하고, 오차가 큰 데이터 포인트에 가중치를 부여한다.
- 다음 모델은 가중치가 부여된 데이터를 사용하여 학습한다
- 이 과정을 반복하여 최종 예측 만든다
## 요약
- 여러 모델의 예측 결과를 결합하여 최종 예측을 만듬
- 모델 개별의 성능을 보완하고 예측 성능을 향상 시킴
- 배깅, 부스팅, 스태킹, 다수결 투표와 같은 다양한 앙상블 기법 존재

In [20]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import VotingClassifier

# 데이터 로드
fish = pd.read_csv(r'https://bit.ly/fish_csv')
fish_target = fish['Species'].to_numpy()
fish_input = fish[['Weight', 'Length', 'Diagonal', 'Height', 'Width']].to_numpy()

# 데이터 분할
train_input, test_input, train_target, test_target = train_test_split(fish_input, fish_target, random_state=42)

# 데이터 스케일링
ss = StandardScaler()
ss.fit(train_input)
train_scaled = ss.transform(train_input)
test_scaled = ss.transform(test_input)

In [5]:
fish.head()

,Species,Weight,Length,Diagonal,Height,Width
0,Bream,242.0,25.4,30.0,11.5200,4.0200
1,Bream,290.0,26.3,31.2,12.4800,4.3056
2,Bream,340.0,26.5,31.1,12.3778,4.6961
3,Bream,363.0,29.0,33.5,12.7300,4.4555
4,Bream,430.0,29.0,34.0,12.4440,5.1340


In [21]:
# 개별 모델 정의 - 3가지 사용
knn_model = KNeighborsClassifier(n_neighbors=3)
svm_model = SVC(kernel='linear', probability=True)
logistic_model = LogisticRegression(max_iter=200)

In [22]:
# 앙상블 모델 정의Voting Classifier
# VotingClassifier를 사용하여 개별 모델들을 앙상블로 결합
# voting = soft는 각 모델의 예측 확률을 평균 내어 최종 예측
voting_model = VotingClassifier(
    estimators=[('knn', knn_model), ('svm', svm_model), ('logistic', logistic_model)],
    voting='soft'  # soft voting 사용
)

In [23]:
# 앙상블 모델 훈련
voting_model.fit(train_scaled, train_target)

# 예측
voting_y_pred = voting_model.predict(test_scaled)

# 정확도 평가
print(f"Voting Classifier Accuracy: {accuracy_score(test_target, voting_y_pred):.2f}")

Voting Classifier Accuracy: 0.88


KNN 개별 모델 정확도

In [25]:
knn_model = KNeighborsClassifier(n_neighbors=3)
knn_model.fit(train_input, train_target)
knn_y_pred = knn_model.predict(test_input)
print(f"KNN Accuracy: {accuracy_score(test_target, knn_y_pred):.2f}")

KNN Accuracy: 0.47


SVM 개별 모델 정확도

In [26]:
svm_model = SVC(kernel='linear')
svm_model.fit(train_input, train_target)
svm_y_pred = svm_model.predict(test_input)
print(f"SVM Accuracy: {accuracy_score(test_target, svm_y_pred):.2f}")

SVM Accuracy: 0.93


로지스틱 회귀 개별 모델 정확도

In [27]:
logistic_model = LogisticRegression(max_iter=200)
logistic_model.fit(train_input, train_target)
logistic_y_pred = logistic_model.predict(test_input)
print(f"Logistic Regression Accuracy: {accuracy_score(test_target, logistic_y_pred):.2f}")


Logistic Regression Accuracy: 0.90


c:\Users\utw09\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


## 앙상블 학습의 장점
1. 안정성
- 앙상블 모델은 여러 모델의 예측을 결합하여 예측의 변동성을 줄인다.
- 따라서 개별 모델 하나만 사용할 때보다 더 안정적인 성능을 보임
2. 일반화 능력
- 다양한 모델을 결합함으로써 과적합overfitting을 방지
- 새로운 데이터에 대한 일반화 성능을 향상시킬 수 있음
3. 상호 보완
- 각 모델이 다른 강점과 약점을 가지기 때문에, 이를 결합함으로써 단일 모델의 약점을 보완할 수 있음
## 앙상블 학습의 단점
1. 복잡성 증가
- 여러 모델을 결합하기 때문에 모델의 구조가 복잡해지고, 이해하고 해석하기 어려움
- 예를 들어, 단일 모델의 경우 어떤 특징이 중요한지 파악하기 쉬운데, 앙상블 모델은 여러 모델의 결과를 결합하므로 이러한 해석이 어렵
2. 계산 비용
- 앙상블 모델은 여러 개의 개별 모델을 학습시키고 예측을 수행해야 하므로, 학습 시간과 계산 비용이 증가
- 특히, 많은 수의 개별 모델을 사용하는 배깅이나 부스팅 같은 방법은 리소스가 많이 필요
3. 과적합 가능성
- 잘못 구성된 앙상블 모델은 오히려 과적합overfitting을 초래
- 이는 특히 개별 모델들이 서로 매우 유사한 경우 발생
- 예를 들어, 배깅Bagging 방법에서 사용되는 개별 결정 트리들이 매우 깊은 경우, 앙상블 모델이 과적합될 가능성이 있음
4. 설정과 튜닝의 어려움
- 앙상블 모델은 여러 하이퍼파라미터를 조정해야 하므로, 적절한 설정을 찾는 과정이 복잡하고 시간이 걸림
- 특히, 부스팅Boosting 같은 방법은 학습률learning rate, 약한 학습기weak learner의 수 등 여러 하이퍼파라미터를 튜닝
5. 디버깅 어려움
- 개별 모델에서 발생하는 문제를 추적하고 해결하는 것이 상대적으로 간단한 반면, 앙상블 모델에서는 개별 모델들이 상호작용하는 방식 때문에 문제의 원인을 파악하기 어려움
- 이는 특히 예측 성능이 기대에 미치지 못할 때 문제를 해결하는 데 어려움
